# Anticipez les besoins en consommation de bâtiments - *Notebook prediction*

## Mission

Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près à la consommation et aux émissions des bâtiments non destinés à l’habitation.

Des relevés minutieux ont été effectués par les agents de la ville en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l’habitation pour lesquels elles n’ont pas encore été mesurées.

Votre prédiction se basera sur les données structurelles des bâtiments (taille et usage des bâtiments, date de construction, situation géographique, ...)

Vous cherchez également à évaluer l’intérêt de l’ENERGY STAR Score pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe. Vous l'intégrerez dans la modélisation et jugerez de son intérêt.

Vous sortez tout juste d’une réunion de brief avec votre équipe. Voici un récapitulatif de votre mission :


1) Réaliser une courte analyse exploratoire.
2) Tester différents modèles de prédiction afin de répondre au mieux à la problématique.

Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).

Mets en place une évaluation rigoureuse des performances, et optimise les hyperparamètres et le choix d’algorithmes de ML à l’aide d’une validation croisée. Tu testeras au minimum 4 algorithmes de famille différente (par exemple : ElasticNet, SVM, GradientBoosting, RandomForest).

In [98]:
import numpy as np

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
import scipy
from scipy import stats
import scipy.stats as st

import statsmodels
import statsmodels.api as sm
import missingno as msno

import sklearn
from sklearn.experimental import enable_iterative_imputer  # Nécessaire pour activer IterativeImputer
from sklearn.impute import IterativeImputer

from sklearn.impute import KNNImputer
# Encodage des variables catégorielles avant d'utiliser KNNImputer
from category_encoders.ordinal import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# pour le centrage et la réduction
from sklearn.preprocessing import StandardScaler
# pour l'ACP
from sklearn.decomposition import PCA

from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix, mean_squared_error, make_scorer, r2_score, mean_absolute_error

from sklearn import dummy
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor

from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

from sklearn.svm import LinearSVC
from sklearn.svm import SVR

from sklearn import kernel_ridge

from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier

import tensorflow
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

import timeit
import warnings

print("numpy version", np.__version__)
print("pandas version", pd.__version__)
print("matplotlib version", matplotlib.__version__)
print("seaborn version", sns.__version__)
print("scipy version", scipy.__version__)
print("statsmodels version", statsmodels.__version__)
print("missingno version", msno.__version__)

print("sklearn version", sklearn.__version__)
print("tensorflow version", tensorflow.__version__)

pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

numpy version 1.26.4
pandas version 2.1.4
matplotlib version 3.8.0
seaborn version 0.13.2
scipy version 1.11.4
statsmodels version 0.14.0
missingno version 0.5.2
sklearn version 1.2.2
tensorflow version 2.18.0


## 1 - Développement et simulation du premier modèle (cible = SiteEnergyUseWN(kBtu))

In [5]:
# Charger le fichier de données
data_fe1 = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_4/data_projet/cleaned/2016_Building_Energy_Benchmarking_fe1.csv", sep=',', low_memory=False)
data_fe1.head()

,NumberofBuildings,NumberofFloors,PropertyGFATotal,SiteEnergyUseWN(kBtu),TotalGHGEmissions,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_BALLARD,Neighborhood_Ballard,Neighborhood_CENTRAL,Neighborhood_Central,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_Delridge,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_North,Neighborhood_Northwest,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",electricity_percent,gaz_percent,steam_percent
0,1.0,12,88434.0,7456910.0,249.98,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.61,17.66,27.73
1,1.0,11,103566.0,8664479.0,295.86,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,38.66,61.34,0.00
2,1.0,10,61320.0,6946800.5,286.43,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.75,26.66,32.59
3,1.0,18,175580.0,14656503.0,505.01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.88,62.12,0.00
4,1.0,2,97288.0,12581712.0,301.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,60.99,39.01,0.00


In [6]:
data_fe1.shape

(1444, 59)

### 1.1 - Sélectionner les features et la cible :

In [8]:
y_fe1_conso = data_fe1['SiteEnergyUseWN(kBtu)']
X_fe1 = data_fe1.drop('SiteEnergyUseWN(kBtu)', axis=1, inplace=False)
X_fe1.shape

(1444, 58)

In [9]:
y_fe1_conso.shape

(1444,)

In [10]:
y_fe1_emissions = data_fe1['TotalGHGEmissions']
X_fe1 = X_fe1.drop('TotalGHGEmissions', axis=1, inplace=False)
X_fe1.shape

(1444, 57)

In [11]:
y_fe1_emissions.shape

(1444,)

In [12]:
X_fe1.head()

,NumberofBuildings,NumberofFloors,PropertyGFATotal,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_BALLARD,Neighborhood_Ballard,Neighborhood_CENTRAL,Neighborhood_Central,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_Delridge,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_North,Neighborhood_Northwest,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",electricity_percent,gaz_percent,steam_percent
0,1.0,12,88434.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.61,17.66,27.73
1,1.0,11,103566.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,38.66,61.34,0.00
2,1.0,10,61320.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.75,26.66,32.59
3,1.0,18,175580.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37.88,62.12,0.00
4,1.0,2,97288.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,60.99,39.01,0.00


### 1.2 - Standardiser les valeurs et créer les jeux d'entraînement / test

In [14]:
X_scale_fe1 = StandardScaler().fit_transform(X_fe1)
#X_scale_fe1 = std_scale.transform(X_fe1)

In [15]:
df = pd.DataFrame(X_scale_fe1)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
count,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1444.000000,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1.444000e+03,1444.000000,1.444000e+03,1.444000e+03,1.444000e+03
mean,1.796039e-16,4.920656e-17,-1.180957e-16,-2.952394e-17,1.230164e-17,-5.904787e-17,-4.428590e-17,-2.583344e-17,0.000000,2.706361e-17,5.535738e-18,2.952394e-17,-4.920656e-18,-6.888918e-17,-4.920656e-18,2.460328e-17,-1.476197e-17,1.230164e-17,-3.936525e-17,3.936525e-17,1.968262e-17,2.952394e-17,3.690492e-18,1.968262e-17,1.476197e-17,2.460328e-17,4.920656e-18,-4.920656e-17,-1.476197e-17,2.460328e-17,-2.460328e-18,-3.936525e-17,-1.722230e-17,-1.968262e-17,-1.082544e-16,-2.460328e-17,-1.722230e-17,2.460328e-17,1.476197e-17,4.428590e-17,-4.920656e-18,-2.460328e-17,-4.674623e-17,3.567476e-17,-2.460328e-17,-3.444459e-17,2.460328e-18,-3.936525e-17,-7.873050e-17,-8.365115e-17,1.476197e-17,2.952394e-17,-6.888918e-17,0.000000,3.887318e-16,3.936525e-17,-9.841312e-18
std,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346e+00,1.000346,1.000346e+00,1.000346e+00,1.000346e+00
min,-1.330380e-01,-8.213385e-01,-8.378769e-01,-1.913417e-01,-2.632491e-02,-2.100167e-01,-3.051414e-01,-5.270463e-02,-0.277143,-3.724195e-02,-1.406201e-01,-2.683503e-01,-4.562771e-02,-4.148228e-01,-8.761395e-02,-1.185114e-01,-9.154173e-02,-2.469505e-01,-1.406201e-01,-1.024541e-01,-4.948006e-01,-1.643990e-01,-1.024541e-01,-3.845175e-01,-2.205822e-01,-2.063959e-01,-6.459460e-02,-1.751924e-01,-5.270463e-02,-1.687899e-01,-2.632491e-02,-4.828396e-01,-4.562771e-02,-2.771432e-01,-5.410623e-01,-3.037835e-01,-3.145386e-01,-1.989895e-01,-2.871564e-01,-2.324104e-01,-7.919455e-02,-5.270463e-02,-1.751924e-01,-1.643990e-01,-3.417348e-01,-2.913736e-01,-3.237595e-01,-1.854291e-01,-3.492694e-01,-4.330127e-01,-3.604032e-01,-3.467683e-01,-3.542416e-01,-0.311872,-2.600511e+00,-1.092064e+00,-2.018262e-01
25%,-1.330380e-01,-5.543388e-01,-5.937670e-01,-1.913417e-01,-2.632491e-02,-2.100167e-01,-3.051414e-01,-5.270463e-02,-0.277143,-3.724195e-02,-1.406201e-01,-2.683503e-01,-4.562771e-02,-4.148228e-01,-8.761395e-02,-1.185114e-01,-9.154173e-02,-2.469505e-01,-1.406201e-01,-1.024541e-01,-4.948006e-01,-1.643990e-01,-1.024541e-01,-3.845175e-01,-2.205822e-01,-2.063959e-01,-6.459460e-02,-1.751924e-01,-5.270463e-02,-1.687899e-01,-2.632491e-02,-4.828396e-01,-4.562771e-02,-2.771432e-01,-5.410623e-01,-3.037835e-01,-3.145386e-01,-1.989895e-01,-2.871564e-01,-2.324104e-01,-7.919455e-02,-5.270463e-02,-1.751924e-01,-1.643990e-01,-3.417348e-01,-2.913736e-01,-3.237595e-01,-1.854291e-01,-3.492694e-01,-4.330127e-01,-3.604032e-01,-3.467683e-01,-3.542416e-01,-0.311872,-8.035571e-01,-1.092064e+00,-2.018262e-01
5

In [16]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scale_fe1, y_fe1_conso, test_size=0.25, random_state=42 ) # 25% des données dans le jeu de test

In [17]:
X_train.shape

(1083, 57)

In [18]:
X_test.shape

(361, 57)

In [19]:
y_train.shape

(1083,)

In [20]:
y_test.shape

(361,)

### 1.3 - Premiers modèles sans validation croisée

Il s'agit d'évaluer quelques modèles sans utiliser la validation croisée, en partant d'une baseline, pour aller vers des modèles plus élaborés.

L'hyperparamètre "alpha" sera fixe dans un premier temps (pas de validation croisée pour optimiser l'hyperparamètre).

On fera une boucle sur chaque modèle, et on stockera les scrores dans un tableau. Mais d'abord créons les fonctions qui seront utlisées dans la boucle :

**Baseline avec DummyRegressor**

On va utiliser la stratégie de la moyenne : prédit la moyenne des valeurs cibles d'entraînement. Créons la fonction qui prend les jeux d'entraînement et de tests et en entrée, et retourne les scores MSE, RMSE, R2, MAE:

In [23]:
def fit_dummyRegressor(X_train, y_train, X_test, y_test):

    start_time = timeit.default_timer()
    
    # Initialisation du DummyRegressor avec la stratégie 'mean'
    dummy_regressor = DummyRegressor(strategy='mean')
    
    # Entraînement du modèle
    dummy_regressor.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = dummy_regressor.predict(X_test)

    elapsed = timeit.default_timer() - start_time
    
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle de régression Ridge**

La régression ridge nous permet de réduire l'amplitude des coefficients d'une régression linéaire et d'éviter le sur-apprentissage. On optimisera l'hyperparamètre lors de la validation croisée avec GridSearchCV.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [25]:
def fit_ridge(X_train, y_train, X_test, y_test, alpha):

    start_time = timeit.default_timer()
    
    # Initialisation du modèle Ridge avec un paramètre alpha
    ridge_regressor = Ridge(alpha=alpha)  # alpha contrôle la régularisation ; plus grand, plus de régularisation
    
    # Entraînement du modèle
    ridge_regressor.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = ridge_regressor.predict(X_test)

    elapsed = timeit.default_timer() - start_time
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle de régression Lasso**

Le Lasso est une méthode de sélection de variables et de réduction de dimension supervisée : les variables qui ne sont pas nécessaires à la prédiction de l'étiquette sont éliminées.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [27]:
def fit_lasso(X_train, y_train, X_test, y_test, alpha):

    start_time = timeit.default_timer()
    
    # Initialisation du modèle Lasso avec un paramètre alpha
    lasso_regressor = Lasso(alpha=alpha)  # alpha contrôle la régularisation ; plus grand, plus de régularisation
    
    # Entraînement du modèle
    lasso_regressor.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = lasso_regressor.predict(X_test)

    elapsed = timeit.default_timer() - start_time
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle ElasticNet**

La méthode elastic net qui combine les deux termes de régularisation en un (Ridge L2 et Lasso L1).

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [50]:
def fit_elasticNet(X_train, y_train, X_test, y_test, alpha):

    start_time = timeit.default_timer()

    # Initialisation du modèle ElasticNet
    elastic_net = ElasticNet(
        alpha=alpha, 
        l1_ratio=0.5, # Mélange égal entre ridge et lasso
        max_iter=1000, random_state=42
    )

    # Entraînement du modèle
    elastic_net.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = elastic_net.predict(X_test)
    
    elapsed = timeit.default_timer() - start_time
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Bagging - Modèle RamdonForestRegressor**

Appelé aussi le bagging qui, appliqué aux arbres de décision, donne naissance au modèle de forêt aléatoire. 

Une forêt aléatoire est un ensemble de nombreux arbres de décision qui sont combinés pour produire une prédiction plus précise et plus robuste. Chaque arbre de décision est construit à partir d'un échantillon aléatoire des données et les résultats sont moyennés pour obtenir la prédiction finale. 

Le modèle de forêt aléatoire est intrinsèquement parallèle. Les arbres sont entraînés en même temps sur des parties du dataset.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [30]:
def fit_ramdomForestRegressor(X_train, y_train, X_test, y_test, X_fe1):

    start_time = timeit.default_timer()
    
    # Création du modèle
    # n_estimators : Nombre d'arbres dans la forêt. Valeur par défaut = 100. Une valeur plus élevée peut améliorer la précision mais augmente le temps de calcul.
    model = RandomForestRegressor(n_estimators=100, random_state=42)  # random_state permet
    
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred = model.predict(X_test)

    elapsed = timeit.default_timer() - start_time
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    # Afficher l'importance des features
    print("Importance des features dans le RandomForestRegressor :")
    importances = model.feature_importances_
    # Création d'un DataFrame pour afficher l'importance des features
    feature_importance = pd.DataFrame({'Feature': X_fe1.columns, 'Importance': importances})
    # Tri par ordre d'importance décroissante
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

    # Afficher les résultats
    print(feature_importance)

    return mse, rmse, r2, mae, elapsed

**Boosting - Modèle GradientBoostingRegressor**

Le boosting enchaîne l'entraînement des prédicteurs faibles de façon séquentielle, en se concentrant à chaque itération sur les échantillons qui ont généré le plus d'erreurs.

Il s'agit d'une méthode d'ensemble qui construit un modèle prédictif puissant en combinant plusieurs modèles faibles (typiquement des arbres de décision) de manière séquentielle

Le modèle de forêt aléatoire est intrinsèquement parallèle. Les arbres sont entraînés en même temps sur des parties du dataset.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [32]:
def fit_gradientBoostingRegressor(X_train, y_train, X_test, y_test, X_fe1):

    start_time = timeit.default_timer()

    # Création du modèle de régression Gradient Boosting
    gb_regressor = GradientBoostingRegressor(
        n_estimators=100,  # Nombre d'arbres
        learning_rate=0.1,  # Taux d'apprentissage (ici valeur par défaut)
        max_depth=3,  # Profondeur maximale des arbres (ici valeur par défaut)
        random_state=42  # Pour la reproductibilité
    )

    # Entraînement du modèle
    gb_regressor.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = gb_regressor.predict(X_test)

    elapsed = timeit.default_timer() - start_time
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle SVR**

Pour effectuer une régression avec un SVM (Support Vector Machine), on utilise le modèle appelé Support Vector Regression (SVR), qui fait partie des algorithmes de régression basés sur les SVM.

Le SVR cherche à trouver une fonction qui ne s'écarte pas trop des valeurs cibles, avec un contrôle sur la marge d'erreur autorisée.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [102]:
def fit_SVR(X_train, y_train, X_test, y_test):
    
    start_time = timeit.default_timer()
    
    # Initialisation du modèle SVR
    svr_model = SVR(
        kernel='rbf', 
        C=1.0,  # Paramètre de régularisation qui contrôle la pénalité pour les erreurs. Un C élevé cherche à minimiser les erreurs.
        epsilon=0.1 # Contrôle la largeur de la marge autour de la fonction cible. Les points situés dans cette marge ne contribuent pas à la fonction de coût
    )

    # Entraînement du modèle SVR
    svr_model.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = svr_model.predict(X_test)

    elapsed = timeit.default_timer() - start_time
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**1ere itération des modèles pour différentes valeurs d'alpha**

Créons une fonction qui permet de :
- lancer tous les modèles précédentsn avec plusieurs valeurs d'alpha pour Ridge, Lasso et ElasticNet
- afficher les scores

In [104]:
warnings.filterwarnings("ignore")

mse, rmse, r2, mae, elapsed = fit_dummyRegressor(X_train, y_train, X_test, y_test)
scores_array = np.array([['DummyRegressor', '', mse, rmse, r2, mae, elapsed]])

alphas = np.logspace(-6, 6, 7)

for alpha in alphas:
    mse, rmse, r2, mae, elapsed = fit_ridge(X_train, y_train, X_test, y_test, alpha)
    scores_array = np.vstack([scores_array, ['ridge', alpha, mse, rmse, r2, mae, elapsed]])
    mse, rmse, r2, mae, elapsed = fit_lasso(X_train, y_train, X_test, y_test, alpha)
    scores_array = np.vstack([scores_array, ['lasso', alpha, mse, rmse, r2, mae, elapsed]])
    mse, rmse, r2, mae, elapsed = fit_elasticNet(X_train, y_train, X_test, y_test, alpha)
    scores_array = np.vstack([scores_array, ['elasticNet', alpha, mse, rmse, r2, mae, elapsed]])

mse, rmse, r2, mae, elapsed = fit_ramdomForestRegressor(X_train, y_train, X_test, y_test, X_fe1)
scores_array = np.vstack([scores_array, ['RamdomForestRegressor', '', mse, rmse, r2, mae, elapsed]])

mse, rmse, r2, mae, elapsed = fit_gradientBoostingRegressor(X_train, y_train, X_test, y_test, X_fe1)
scores_array = np.vstack([scores_array, ['gradientBoostingRegressor', '', mse, rmse, r2, mae, elapsed]])

mse, rmse, r2, mae, elapsed = fit_SVR(X_train, y_train, X_test, y_test)
scores_array = np.vstack([scores_array, ['SVR', '', mse, rmse, r2, mae, elapsed]])

print(scores_array)

Importance des features dans le RandomForestRegressor :
                                            Feature  Importance
2                                  PropertyGFATotal    0.504276
21  PrimaryPropertyType_Supermarket / Grocery Store    0.087605
54                              electricity_percent    0.065398
1                                    NumberofFloors    0.048745
55                                      gaz_percent    0.037337
23                    PrimaryPropertyType_Warehouse    0.033062
13                        PrimaryPropertyType_Other    0.024428
7                    PrimaryPropertyType_Laboratory    0.017801
16                   PrimaryPropertyType_Restaurant    0.012443
19        PrimaryPropertyType_Senior Care Community    0.011961
52                   YearBuilt_Bin_(1992.0, 2003.5]    0.009903
50                   YearBuilt_Bin_(1969.0, 1980.5]    0.008951
11           PrimaryPropertyType_Mixed Use Property    0.008767
31                            Neighborhood_DOWNT

In [106]:
# Conversion de l'array en DataFrame
df_scores_fe1 = pd.DataFrame(scores_array, columns=['Modèle', 'Alpha', 'MSE', 'RMSE', 'R2', 'MAE', 'TIME'])

# on transforme la colonne R2 en numérique
df_scores_fe1['R2'] = pd.to_numeric(df_scores_fe1['R2'], errors='coerce')

# On trie le dataframe sur la colonne R2 du pmus grand au plus petit
df_scores_fe1.sort_values(by='R2', ascending=False, inplace=True)

df_scores_fe1.head(30)

,Modèle,Alpha,MSE,RMSE,R2,MAE,TIME
17,lasso,10000.0,4832927251745.98,2198391.97,0.66,1512727.73,0.0064317000001210545
9,elasticNet,0.01,4854027003860.09,2203185.65,0.66,1522391.65,0.06852179999987129
11,lasso,1.0,4856086661077.91,2203653.03,0.65,1519721.86,0.08146110000006956
2,lasso,1e-06,4856086414716.8,2203652.97,0.65,1519720.94,0.11726810000027399
3,elasticNet,1e-06,4856087747821.28,2203653.27,0.65,1519721.83,0.09174589999975069
4,ridge,0.0001,4858029990853.7,2204093.92,0.65,1520444.52,0.0022143000001051405
5,lasso,0.0001,4856086414741.06,2203652.97,0.65,1519720.94,0.0728836000002957
6,elasticNet,0.0001,4856212071213.06,2203681.48,0.65,1519805.93,0.07260150000001886
7,ridge,0.01,4857923055464.96,2204069.66,0.65,1520443.58,0.0018106999996234663
8,lasso,0.01,4856086417168.17,2203652.97,0.65,1519720.95,0.073098200000004


Sans surprise, le  modèle DummyRegressor a des mauvais scores, comme les modèles Ridge avec alpha > 1000, et le lasso avec alpha très élevé.

Le Lasso est le meilleur modèle pour le moment.

On observe également que pour le RandomForestRegressor, la variable PropertyGFATotal à la plus grande importance, et de loin.

In [37]:
STOP

NameError: name 'STOP' is not defined

L'utilisation de GridSearchCV avec une régression Ridge permet d'optimiser les hyperparamètres du modèle, en particulier le paramètre de régularisation alpha. GridSearchCV effectue une recherche exhaustive sur un ensemble de valeurs d'hyperparamètres spécifiés, en combinant ces valeurs avec la validation croisée pour évaluer chaque combinaison. Cela permet de trouver les meilleurs hyperparamètres pour le modèle.

In [ ]:
# Choisir un score à optimiser, ici le MSE
scoring = {
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
    'R2': 'r2'
}

In [ ]:
# Définir une grille de paramètres à tester
param_grid = {
    'alpha': np.logspace(-6, 6, 13)
}

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
grid_search = model_selection.GridSearchCV(
    estimator=Ridge(),           # une régression Ridge
    param_grid = param_grid,     # hyperparamètres à tester
    cv=5,                        # nombre de folds de validation croisée
    scoring=scoring,             # score à optimiser
    refit='R2'                  # score utilisé pour le choix de l'hyperparamètre
)
    
# Optimiser ce grid_search sur le jeu d'entraînement
grid_search.fit(X_train, y_train)
    
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(grid_search.best_params_)

# Afficher le meilleur score
print("Meilleu(s) score sur le jeu d'entraînement:")
print(grid_search.best_score_)

# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for score_name in scoring.keys():
   print(f"\nScores pour '{score_name}':")
   for mean, std, params in zip(
            grid_search.cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
            grid_search.cv_results_[f'std_test_{score_name}'],   # écart-type du score
            grid_search.cv_results_['params']                    # valeur de l'hyperparamètre
    ):
       print(f"{score_name} = {mean:.3f} (+/-{std * 2:.03f}) for {params}")

Pour la valeur d'alpha = 1.0, le MSE est meilleur que le modèle de régression classique et la baseline avec Dummy.

Scores sur les prédictions avec les meilleurs hyperparamètres sur le fichier de test :

In [ ]:
y_pred = grid_search.predict(X_test)

# Calculer des métriques pour évaluer le modèle
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE sur le test: {rmse:.3f}")
print(f"MSE sur le test : {mse:.3f}")
print(f"R^2 sur le test : {r2:.3f}")

Si on regarde la valeur de R2, les prédictions sont meilleures sur le fichier de test.

### 1.5 - Modèle de régression Lasso en validation croisée

Le Lasso est une méthode de sélection de variables et de réduction de dimension supervisée : les variables qui ne sont pas nécessaires à la prédiction de l'étiquette sont éliminées.

On va utliser GridSearchCV avec l'estimateur Lasso()

In [ ]:
warnings.filterwarnings("ignore")

# Définir une grille de paramètres à tester
param_grid = {
    'alpha': np.logspace(-5, 1, 50)
}

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
grid_search = model_selection.GridSearchCV(
    estimator=Lasso(),             # une régression Lasso
    param_grid = param_grid,     # hyperparamètres à tester
    cv=5,                        # nombre de folds de validation croisée
    scoring=scoring,             # score à optimiser
    refit='R2'                  # score utilisé pour le choix de l'hyperparamètre
)
    
# Optimiser ce grid_search sur le jeu d'entraînement
grid_search.fit(X_train, y_train)
    
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(grid_search.best_params_)

# Afficher le meilleur score
print("Meilleu(s) score sur le jeu d'entraînement:")
print(grid_search.best_score_)

# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for score_name in scoring.keys():
   print(f"\nScores pour '{score_name}':")
   for mean, std, params in zip(
            grid_search.cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
            grid_search.cv_results_[f'std_test_{score_name}'],   # écart-type du score
            grid_search.cv_results_['params']                    # valeur de l'hyperparamètre
    ):
       print(f"{score_name} = {mean:.3f} (+/-{std * 2:.03f}) for {params}")

Le résultat du modèle Lasso est comparable à celui du modèle Ridge. 

Scores sur le fichier de test :

In [ ]:
y_pred = grid_search.predict(X_test)

# Calculer des métriques pour évaluer le modèle
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE sur le test: {rmse:.3f}")
print(f"MSE sur le test : {mse:.3f}")
print(f"R^2 sur le test : {r2:.3f}")

In [ ]:
FAIRE UNE BOUCLE SUR LES MODELES, ET AJOUTER LES AUTRES MODELES : random forest, light gbm, xgboost, gradientboosting regressor, elastic net
AJOUTER LES AUTRES METRICS MAE ET RMSE.